# NDVI as indicator of degradation of forest

In [1]:
import ee
import geemap

## Create an interactive map

In [2]:
ee.Initialize()
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Define a region of interest (ROI)

In [3]:
roi = ee.Geometry.Point(-62.9287, -10.4480)
Map.addLayer(roi, {}, 'roi')

## Filter ImageCollection and dates

In [5]:
start_date = '2019-01-30'
end_date = '2019-12-31'

l8 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)

In [6]:
print(l8.size().getInfo())

26


## Create a median composite

In [7]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')

## Define functions to add time bands

In [8]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [9]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [10]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [11]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

## Map over an ImageCollection

In [12]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

## Create a quality mosaic

In [13]:
greenest = withNDVI.qualityMosaic('NDVI')

## Display the max value band

In [14]:
ndvi = greenest.select('NDVI')
palette = ['red', 'green', 'yellow','yellow']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [15]:
Map

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(Togg…